## Configuration

The default behaviour of MonarqDevice's transpiler goes as such :

1. Decompose measures if they are not in the computational basis
2. Decompose 3+ qubits gates and non-standard gates to a subset that is easier to optimize (clifford+t gates)
2. Map the circuit's wires to the machines qubits by using the circuit's and machine's connectivity graphs
3. If any 2+ qubit gates are not connected in given mapping, connect them using swaps
4. Optimize the circuit (commute gates, merge rotations, remove inverses and trivial gates)
5. Decompose non-native gates to native ones

monarq.default has a default configuration, so you don't have to set one by yourself. 

In [ ]:
from pennylane_calculquebec.API.client import CalculQuebecClient
import pennylane as qml

# change the information in the client for your credentials
my_client = CalculQuebecClient(
    host="your host",
    user="your user",
    access_token="your access token",
    project_id="your project id",
)

# no config passed in arguments = config = MonarqDefaultConfig
dev = qml.device("monarq.default", client=my_client)

You can change the behaviour of the transpiler by passing a config object to the device. Tread carefully while using this feature because it could lead to your circuit not being run.

Here's how to do it :


1. import the config and classes that interest you:

In [2]:
from pennylane_calculquebec.processing.config import (
    MonarqDefaultConfig,
    ProcessingConfig,
)
from pennylane_calculquebec.processing.steps import (
    CliffordTDecomposition,
    ASTAR,
    Swaps,
    IterativeCommuteAndMerge,
    MonarqDecomposition,
    DecomposeReadout,
)

For simplicity, we'll store the transpiling steps in variables : 

In [3]:
decomp_readout = DecomposeReadout()
base_decomp = CliffordTDecomposition()
placement = ASTAR("yamaska")
routing = Swaps("yamaska")
optimization = IterativeCommuteAndMerge()
native_decomp = MonarqDecomposition()

2. create a config object and set the behaviour you want

In [4]:
# this is the default configuration (it is used by default in monarq.default)
default_config = MonarqDefaultConfig("yamaska")

# This is an equivalent config, built up by hand using the TranspilerConfig base class
default_config = ProcessingConfig(
    decomp_readout, base_decomp, placement, routing, optimization, native_decomp
)

# Say you are confident that your circuit is well placed and routed, you can skip those steps by creating a custom config, and not using placement / routing steps
custom_config = ProcessingConfig(
    decomp_readout, base_decomp, optimization, native_decomp
)

3. pass the config object to the device, along with the client and other arguments

In [5]:
dev = qml.device("monarq.default", client=my_client, processing_config=custom_config)

There are a set of preset configurations that you can use. Each corresponds to a typical use case that are often met. Here are some of them

In [6]:
from pennylane_calculquebec.processing.config import (
    MonarqDefaultConfig,
    MonarqDefaultConfigNoBenchmark,
    NoPlaceNoRouteConfig,
    EmptyConfig,
)

# this is the preset that is used by default when you dont set a config manually
default = MonarqDefaultConfig("yamaska")

# this preset does the same thing as the default one, without the benchmarking acceptance tests in the placement and routing step
no_benchmark = MonarqDefaultConfigNoBenchmark("yamaska")

# this preset skips the placement and routing part. Could be useful if you want to choose machine qubits and couplers manually
no_place_no_route = NoPlaceNoRouteConfig()

# this preset does nothing. It can be used if your circuit is manually transpiled.
# Make sure your wires respect MonarQ's topology and that your gates are native to MonarQ.
empty = EmptyConfig()

### Benchmarks

The transpiler uses benchmarks in the placement and routing part.
Specifically, the state 1 readout fidelity and the cz gate fidelity are used to measure the qubit and coupler accuracy respectively.
Those values are used to determine if qubits are working, but also to choose the best qubits and couplers for your circuits.

You can set the placement and routing steps not to use benchmarking if you want to. 

To do so, just set ```use_benchmark``` to ```False``` in the MonarqDefaultConfig's constructor

In [7]:
my_config = MonarqDefaultConfig("yamaska", False)

# the former is the same as the following preset :
my_config = MonarqDefaultConfigNoBenchmark("yamaska")

Alternatively, set ```use_benchmark``` to ```False``` in the placement / routing steps

In [9]:
placement_no_benchmark = ASTAR("yamaska", use_benchmark=False)
routing_no_benchmark = Swaps("yamaska", use_benchmark=False)

# you can then create a custom configuration with your custom steps
config_no_benchmark = ProcessingConfig(
    decomp_readout,
    base_decomp,
    placement_no_benchmark,
    routing_no_benchmark,
    optimization,
    native_decomp,
)

Even though there may be scenarios where it is beneficial to bypass the transpiling capabilites of the plugin, it is strongly recommended to stick to the default configuration. 

You don't have to pass any configuration to the device to use the default preset.

In [10]:
dev = qml.device("monarq.default", client=my_client)

# the former is the same as doing the latter

my_config = MonarqDefaultConfig("yamaska")

dev = qml.device("monarq.default", client=my_client, processing_config=my_config)